In [3]:
import pymongo
import json
from pymongo import MongoClient
import pandas as pd

In [7]:
# create a database
client = pymongo.MongoClient("mongodb://localhost:27017")
st_db = client.student
records = st_db.st_details

In [8]:
#load the student.json dataset

with open("students.json") as file:
    stud = [json.loads(i) for i in file]


In [9]:
records.insert_many(stud)

In [10]:
p = records.count_documents({})

In [115]:
for i in records.find():
      print(i.keys)

<built-in method keys of dict object at 0x7ff369c41700>
<built-in method keys of dict object at 0x7ff340042140>
<built-in method keys of dict object at 0x7ff369c3e1c0>
<built-in method keys of dict object at 0x7ff369c3e640>
<built-in method keys of dict object at 0x7ff369c3eb00>
<built-in method keys of dict object at 0x7ff36a33f440>
<built-in method keys of dict object at 0x7ff36a369d80>
<built-in method keys of dict object at 0x7ff36a350680>
<built-in method keys of dict object at 0x7ff36a33d900>
<built-in method keys of dict object at 0x7ff36a33ff00>
<built-in method keys of dict object at 0x7ff36a39b8c0>
<built-in method keys of dict object at 0x7ff36a4c2e40>
<built-in method keys of dict object at 0x7ff36a4ca5c0>
<built-in method keys of dict object at 0x7ff36a4d2980>
<built-in method keys of dict object at 0x7ff36a4da2c0>
<built-in method keys of dict object at 0x7ff36a4e1940>
<built-in method keys of dict object at 0x7ff36a4e6340>
<built-in method keys of dict object at 0x7ff36a

#### 1. Find the student name who scored maximum scores in all (exam, quiz and homework)?

In [146]:
max_mark = records.aggregate([ 
    {"$unwind":"$scores"},
    {"$group":
     {"_id":"$_id",
      "name":{"$first":"$name"},"Type":{"$push":"$scores.type"},
      "high_score":{"$sum":"$scores.score"}}},
    {"$sort":{"high_score":-1}},
    {"$limit":1}
        ])

for i in max_mark:
    print(i)   

{'_id': 13, 'name': 'Jessika Dagenais', 'Type': ['exam', 'quiz', 'homework'], 'high_score': 275.94947751518237}


####  2. Find students who scored below average in the exam and pass mark is 40%?


In [147]:
below_avg = records.aggregate([
    {"$unwind":"$scores"},
    {"$match":{'scores.type':'exam',"scores.score":{"$gt":40,"$lt":60}}
    }
      
])
for i in below_avg:
  print(i)

{'_id': 5, 'name': 'Wilburn Spiess', 'scores': {'score': 44.87186330181261, 'type': 'exam'}}
{'_id': 10, 'name': 'Denisha Cast', 'scores': {'score': 45.61876862259409, 'type': 'exam'}}
{'_id': 12, 'name': 'Quincy Danaher', 'scores': {'score': 54.29841278520669, 'type': 'exam'}}
{'_id': 19, 'name': 'Gisela Levin', 'scores': {'score': 44.51211101958831, 'type': 'exam'}}
{'_id': 20, 'name': 'Tressa Schwing', 'scores': {'score': 42.17439799514388, 'type': 'exam'}}
{'_id': 21, 'name': 'Rosana Vales', 'scores': {'score': 46.2289476258328, 'type': 'exam'}}
{'_id': 23, 'name': 'Tamika Schildgen', 'scores': {'score': 45.65432764125526, 'type': 'exam'}}
{'_id': 29, 'name': 'Gwyneth Garling', 'scores': {'score': 48.36644963899371, 'type': 'exam'}}
{'_id': 42, 'name': 'Kayce Kenyon', 'scores': {'score': 44.62441703708117, 'type': 'exam'}}
{'_id': 45, 'name': 'Terica Brugger', 'scores': {'score': 42.1011312120801, 'type': 'exam'}}
{'_id': 57, 'name': 'Chad Rahe', 'scores': {'score': 40.845720273667

#### 3. Find students who scored below pass mark and assigned them as fail, and above pass mark as pass in all the categories.

In [154]:
mark = records.aggregate([
      { "$unwind": "$scores" },
      {"$project": {"_id":0,"name" :1, "scores.type":1, "scores.score":1, 
                    "result":{ 
                        "$cond": { "if" : { "$gte":["$scores.score",40]}, "then":"pass", "else":"fail"}}}}])
print(list(mark))

[{'name': 'aimee Zank', 'scores': {'score': 1.463179736705023, 'type': 'exam'}, 'result': 'fail'}, {'name': 'aimee Zank', 'scores': {'score': 11.78273309957772, 'type': 'quiz'}, 'result': 'fail'}, {'name': 'aimee Zank', 'scores': {'score': 35.8740349954354, 'type': 'homework'}, 'result': 'fail'}, {'name': 'Aurelia Menendez', 'scores': {'score': 60.06045071030959, 'type': 'exam'}, 'result': 'pass'}, {'name': 'Aurelia Menendez', 'scores': {'score': 52.79790691903873, 'type': 'quiz'}, 'result': 'pass'}, {'name': 'Aurelia Menendez', 'scores': {'score': 71.76133439165544, 'type': 'homework'}, 'result': 'pass'}, {'name': 'Corliss Zuk', 'scores': {'score': 67.03077096065002, 'type': 'exam'}, 'result': 'pass'}, {'name': 'Corliss Zuk', 'scores': {'score': 6.301851677835235, 'type': 'quiz'}, 'result': 'fail'}, {'name': 'Corliss Zuk', 'scores': {'score': 66.28344683278382, 'type': 'homework'}, 'result': 'pass'}, {'name': 'Bao Ziglar', 'scores': {'score': 71.64343899778332, 'type': 'exam'}, 'resul

####  4. Find the total and average of the exam, quiz and homework and store them in a separate collection.

In [164]:
total = st_db.total_collection

In [184]:
avg = records.aggregate([
    {"$unwind":"$scores"},
    {"$group":{"_id":"$_id",
               "name":{"$first":"$name"},
               "Total":{"$sum":"$scores.score"},
                "Average":{"$avg":"$scores.score"}}},
                {"$sort":{"_id":1}}])
avgg = []
for i in avg:
    avgg.append(i)
    print(i)
    

{'_id': 0, 'name': 'aimee Zank', 'Total': 49.11994783171814, 'Average': 16.373315943906046}
{'_id': 1, 'name': 'Aurelia Menendez', 'Total': 184.61969202100374, 'Average': 61.53989734033458}
{'_id': 2, 'name': 'Corliss Zuk', 'Total': 139.61606947126907, 'Average': 46.53868982375636}
{'_id': 3, 'name': 'Bao Ziglar', 'Total': 138.70712252233457, 'Average': 46.23570750744486}
{'_id': 4, 'name': 'Zachary Langlais', 'Total': 203.3963625302728, 'Average': 67.79878751009093}
{'_id': 5, 'name': 'Wilburn Spiess', 'Total': 134.0186975547794, 'Average': 44.67289918492647}
{'_id': 6, 'name': 'Jenette Flanders', 'Total': 147.2203575476617, 'Average': 49.07345251588723}
{'_id': 7, 'name': 'Salena Olmos', 'Total': 229.3959334774732, 'Average': 76.46531115915774}
{'_id': 8, 'name': 'Daphne Zheng', 'Total': 112.71677331754347, 'Average': 37.57225777251449}
{'_id': 9, 'name': 'Sanda Ryba', 'Total': 220.08328118626565, 'Average': 73.36109372875522}
{'_id': 10, 'name': 'Denisha Cast', 'Total': 199.88435728

#### 5. Create a new collection which consists of students who scored below average and above 40% in all the categories.

In [206]:
pass_avg = st_db.belowavg_collection

aggregrate = records.aggregate(
[{"$match": 
   {"$expr": 
     {"$and": 
       [{"$gt": [{"$min": "$scores.score"}, 40]},
         {"$lt": [{"$max": "$scores.score"}, 70]}
        ]
      }
    }
  }])

pass_avg = []
for i in aggregrate:
    pass_avg.append(i)
    print(i)

{'_id': 65, 'name': 'Gena Riccio', 'scores': [{'score': 67.58395308948619, 'type': 'exam'}, {'score': 67.2413500951588, 'type': 'quiz'}, {'score': 42.93471779899529, 'type': 'homework'}], 'Result': {'Result_Exam': ['Pass', 'Pass']}}
{'_id': 85, 'name': 'Rae Kohout', 'scores': [{'score': 63.86894250781692, 'type': 'exam'}, {'score': 55.81549538273672, 'type': 'quiz'}, {'score': 59.13566011309437, 'type': 'homework'}], 'Result': {'Result_Exam': ['Pass', 'Pass']}}
{'_id': 109, 'name': 'Flora Duell', 'scores': [{'score': 40.68238966626067, 'type': 'exam'}, {'score': 46.77972040308903, 'type': 'quiz'}, {'score': 69.29400057020965, 'type': 'homework'}], 'Result': {'Result_Exam': ['Pass', 'Pass']}}
{'_id': 110, 'name': 'Nobuko Linzey', 'scores': [{'score': 67.40792606687442, 'type': 'exam'}, {'score': 58.58331128403415, 'type': 'quiz'}, {'score': 47.44831568815929, 'type': 'homework'}], 'Result': {'Result_Exam': ['Pass', 'Pass']}}
{'_id': 185, 'name': 'Kam Senters', 'scores': [{'score': 49.88

#### 6. Create a new collection which consists of students who scored below the fail mark in all the categories.

In [ ]:
db=client.student_database1
collection=db.student_collection1
a=collection.insert_many(file)
print(a.inserted_ids)

In [ ]:
st_db = client.student
records = st_db.st_details

In [ ]:
collection.aggregate([{"$match":{"scores.type":{"$in":["exam","quiz","homework"]},
                                   "scores.0.score":{"$lt":40},"scores.1.score":{"$lt":40},"scores.2.score":{"$lt":40}}},
                                  {"$group":{"_id":"$_id","Name":{"$push":"$name"},"Type":{"$push":"$scores.type"},
                                            "scores":{"$push":"$scores.score"},
                                            "Remarks":{"$push":"FAIL"}}},
                                            {"$out":{"db":"student_database1","coll":"fail in all category"}}])


In [200]:
fail = st_db.failcol

In [208]:
fail_mark = records.aggregate([
    {"$match":{"scores.type":{"$in":["exam","quiz","homework"]},
               "scores.0.score":{"$lt":40},
               "scores.1.score":{"$lt":40},
               "scores.2.score":{"$lt":40}}},
    {"$group":{"_id":"$_id",
               "Name":{"$push":"$name"},
               "Type":{"$push":"$scores.type"},
                "scores":{"$push":"$scores.score"},
                "Remarks":{"$push":"FAIL"}}},
    ])

failed = []
for i in fail_mark:
    failed.append(i)
    print(i)


{'_id': 0, 'Name': ['aimee Zank'], 'Type': [['exam', 'quiz', 'homework']], 'scores': [[1.463179736705023, 11.78273309957772, 35.8740349954354]], 'Remarks': ['FAIL']}


#### 7. Create a new collection which consists of students who scored above pass mark in all the categories.


In [209]:
newcol = st_db.newcollection1

In [212]:
score = records.aggregate(
[{"$match": 
   {"$expr": 
         {"$gt": [{"$min": "$scores.score"}, 40]}}}])

passed = []

for i in score:
      passed.append(i)
      print(i)
newcol.insert_many(passed)

{'_id': 1, 'name': 'Aurelia Menendez', 'scores': [{'score': 60.06045071030959, 'type': 'exam'}, {'score': 52.79790691903873, 'type': 'quiz'}, {'score': 71.76133439165544, 'type': 'homework'}], 'Result': {'Result_Exam': ['Pass', 'Pass']}}
{'_id': 7, 'name': 'Salena Olmos', 'scores': [{'score': 90.37826509157176, 'type': 'exam'}, {'score': 42.48780666956811, 'type': 'quiz'}, {'score': 96.52986171633331, 'type': 'homework'}], 'Result': {'Result_Exam': ['Pass', 'Pass']}}
{'_id': 10, 'name': 'Denisha Cast', 'scores': [{'score': 45.61876862259409, 'type': 'exam'}, {'score': 98.35723209418343, 'type': 'quiz'}, {'score': 55.90835657173456, 'type': 'homework'}], 'Result': {'Result_Exam': ['Pass', 'Pass']}}
{'_id': 11, 'name': 'Marcus Blohm', 'scores': [{'score': 78.42617835651868, 'type': 'exam'}, {'score': 82.58372817930675, 'type': 'quiz'}, {'score': 87.49924733328717, 'type': 'homework'}], 'Result': {'Result_Exam': ['Pass', 'Pass']}}
{'_id': 12, 'name': 'Quincy Danaher', 'scores': [{'score':

BulkWriteError: batch op errors occurred, full error: {'writeErrors': [{'index': 0, 'code': 11000, 'errmsg': 'E11000 duplicate key error collection: student.newcollection1 index: _id_ dup key: { _id: 1 }', 'keyPattern': {'_id': 1}, 'keyValue': {'_id': 1}, 'op': {'_id': 1, 'name': 'Aurelia Menendez', 'scores': [{'score': 60.06045071030959, 'type': 'exam'}, {'score': 52.79790691903873, 'type': 'quiz'}, {'score': 71.76133439165544, 'type': 'homework'}], 'Result': {'Result_Exam': ['Pass', 'Pass']}}}], 'writeConcernErrors': [], 'nInserted': 0, 'nUpserted': 0, 'nMatched': 0, 'nModified': 0, 'nRemoved': 0, 'upserted': []}